# Controlling Panda in Pybullet

In [1]:
import sys
sys.path.append('../../lib')

import numpy as np
import pybullet as p
import pybullet_data

import time

In [2]:
def get_joint_states(robot_id, dof = 7):
    q = [p.getJointState(robot_id, i)[0] for i in range(dof)]
    dq = [p.getJointState(robot_id, i)[1] for i in range(dof)]
    return q, dq

def get_joint_limits(robot_id, indices):
    lower_limits = []
    upper_limits = []
    for i in indices:
        info = p.getJointInfo(robot_id, i)
        lower_limits += [info[8]]
        upper_limits += [info[9]]
    limits = np.vstack([lower_limits, upper_limits])
    return limits

def set_q(robot_id, joint_indices, q):
    for i in range(len(q)):
        p.resetJointState(robot_id, joint_indices[i], q[i])

def get_max_torques(robot_id, dof=7):
    max_torque = np.array([p.getJointInfo(robot_id,i)[10] for i in range(dof)])
    return max_torque

#### Start Pybullet

In [4]:
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.81)
p.setTimeStep(0.001)

p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

#### Load robot

In [5]:
p.loadURDF('plane.urdf')
urdf_filename = '../data/urdf/frankaemika_new/panda_arm.urdf'
robot_id = p.loadURDF(urdf_filename, useFixedBase=True)
dof = 7
joint_indices = np.arange(dof)
joint_limits = get_joint_limits(robot_id, joint_indices)
max_torque = get_max_torques(robot_id)

#### Enable torque control
Enable torque control at all joints, by setting it in velocity control mode with max_force = 0 (this is very specific to pybullet, it's just like a trick to enable torque control)

In [6]:
mode = p.VELOCITY_CONTROL

for i in range(7):
    p.setJointMotorControl2(bodyIndex=robot_id, jointIndex = i, controlMode=mode,targetVelocity=0.,force=0)    

#### Reset joint angles

In [7]:
for i in range(7):
    p.resetJointState(robot_id, i,0)

#### Example of Simulation (without control)
Without control, the robot will simply fall down due to gravity

In [ ]:
for i in range(10000):
    p.stepSimulation()
    time.sleep(0.001)

## Generic Control Structure

#### Set the robot at initial config

In [9]:
q_init = (joint_limits[0]+joint_limits[1])/2

set_q(robot_id, joint_indices, q_init)

#### Control Loop

In [ ]:
for i in range(1000):
    q, dq = get_joint_states(robot_id) #get current robot config
    
    #****(FILL IN THIS WITH YOUR CONTROLLER)#****
    tau = ..... 
    
    np.clip(tau, -max_torque, max_torque) #limit the torque to the max values
    p.setJointMotorControlArray(robot_id, joint_indices, p.TORQUE_CONTROL, 
                                forces=tau)
    p.stepSimulation()
    time.sleep(0.001)

# Example of Controllers

## 1. Gravity Compensation

The robot will simply stay at the current configuration, because we exert torque that balances the gravity

#### Set init config

In [14]:
q_init = (joint_limits[0]+joint_limits[1])/2
set_q(robot_id, joint_indices, q_init)

#### Control Loop

In [ ]:
for i in range(10000):
    q, dq = get_joint_states(robot_id)
    
    #Get the torque on the joints caused by gravity, by using inverse dynamics
    #In the inverse dynamics, set current velocity = 0 and desired_acceleration = 0
    #And the output will be gravity
    #Inverse dynamics equation: tau = M.ddq_des + C.dq + G
    #ddq_des = desired acceleration, which is zero (we don't want the robot to move)
    G = np.array(p.calculateInverseDynamics(robot_id, q, [0.]*dof, [0.]*dof))
    tau = 1.*G
    p.setJointMotorControlArray(robot_id, joint_indices, p.TORQUE_CONTROL, 
                                forces=tau)
    p.stepSimulation()
    time.sleep(0.001)

#### Tips: try to multiply G by a factor, e.g. from 0.5 to 1.5, and see what happens

## Moving to a target point

#### Set init config

In [16]:
q_init = (joint_limits[0]+joint_limits[1])/2

In [17]:
set_q(robot_id, joint_indices, q_init)

#### Set goal config (random)

In [21]:
q_goal = joint_limits[0] + np.array([0.3, 0.3, 0.2, 0.3, 0.4, 0.5, 0.1])*(joint_limits[1]-joint_limits[0])

In [22]:
set_q(robot_id, joint_indices, q_goal)

#### Interpolate to get reference trajectory

In [40]:
N = 3000
dq = (q_goal-q_init)/N
q_traj = [q_init + dq*i for i in range(N)]

In [41]:
for q in q_traj:
    set_q(robot_id, joint_indices, q)
    time.sleep(0.001)

## 2.  PD control to track q_traj

In [46]:
set_q(robot_id, joint_indices, q_init)

In [47]:
Kp = np.array([300]*4 + [30]*3) #the gain on the first four joints should be larger
Kd = 0.1*np.sqrt(2*Kp)set_q(robot_id,

In [48]:
for i in range(N):
    q, dq = get_joint_states(robot_id)
    error_pos = q_traj[i]-q
    error_vel = np.zeros(dof)-dq #desired velocity = zero. But it can also be computed from q_traj
    tau = Kp*(error_pos) + Kd*(error_vel)
    np.clip(tau, -max_torque, max_torque)
    p.setJointMotorControlArray(robot_id, joint_indices, p.TORQUE_CONTROL, 
                                forces=tau)
    p.stepSimulation()
    time.sleep(0.001)

In [49]:
print(np.array(q)-q_goal)

[-0.0125823  -0.06588684 -0.063441   -0.02716513 -0.04460622 -0.03050568
  0.02163088]


## 3. PD control + Gravity compensation to track q_traj

In [54]:
set_q(robot_id, joint_indices, q_init)

In [55]:
Kp = np.array([30]*4 + [10]*3)
Kd = .1*np.sqrt(2*Kp)

In [56]:
for i in range(N):
    q, dq = get_joint_states(robot_id)
    error_pos = q_traj[i]-q
    error_vel = np.zeros(dof)-dq
    G = np.array(p.calculateInverseDynamics(robot_id, q, [0.]*dof, [0.]*dof))
    tau = G + Kp*(error_pos) + Kd*(error_vel)
    np.clip(tau, -max_torque, max_torque)
    p.setJointMotorControlArray(robot_id, joint_indices, p.TORQUE_CONTROL, 
                                forces=tau)
    p.stepSimulation()
    time.sleep(0.001)

In [57]:
print(np.array(q)-q_goal)

[-0.02087358  0.02960051  0.02455801  0.09394584  0.00554015  0.04019914
  0.03661858]
